In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.pydantic_v1 import BaseModel, Field
from langchain.schema import StrOutputParser

In [1]:
%load_ext dotenv
%dotenv

In [4]:
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
)

In [7]:
class RawQuestion(BaseModel):
    question: str = Field(..., title="Question", description="Question to ask")


class RawQuestionList(BaseModel):
    questions: list[RawQuestion] = Field(
        ..., title="Questions", description="List of questions"
    )


raw_question_list_parser = PydanticOutputParser(pydantic_object=RawQuestionList)

generate_question_prompt = """
Generate a list of {topic} questions that doesn't have a clear answer.
Format each question to be a yes no question.
Example:
1. Does God exist?
2. Does life have meaning?
3. Is morality objective?

{format_instructions}
"""

generate_question_prompt = PromptTemplate(
    template=generate_question_prompt,
    input_variables=["topic"],
    partial_variables={
        "format_instructions": raw_question_list_parser.get_format_instructions(),
    },
)

raw_questions_chain = generate_question_prompt | llm | raw_question_list_parser

topics = ["philosophical", "political", "religious", "ethical", "moral"]

raw_questions = [(raw_questions_chain).invoke({"topic": topic}) for topic in topics]

In [9]:
all_raw_questions = [
    question for topic in raw_questions for question in topic.questions
]
all_raw_questions

[RawQuestion(question='Is reality objective?'),
 RawQuestion(question='Do humans have free will?'),
 RawQuestion(question='Is there an afterlife?'),
 RawQuestion(question='Is there a universal moral code?'),
 RawQuestion(question='Is beauty subjective?'),
 RawQuestion(question='Is time travel possible?'),
 RawQuestion(question='Is there a purpose to the universe?'),
 RawQuestion(question='Is consciousness purely physical?'),
 RawQuestion(question='Is there such a thing as fate?'),
 RawQuestion(question='Is there an ultimate truth?'),
 RawQuestion(question='Is abortion morally wrong?'),
 RawQuestion(question='Should the death penalty be abolished?'),
 RawQuestion(question='Is capitalism the best economic system?'),
 RawQuestion(question='Should the government provide universal healthcare?'),
 RawQuestion(question='Is climate change primarily caused by human activity?'),
 RawQuestion(question='Should marijuana be legalized?'),
 RawQuestion(question='Is affirmative action necessary to add

In [10]:
class YesNoQuestion(BaseModel):
    word: str = Field(
        ..., title="Phrase", description="The phrase the question is asking about"
    )
    question: str = Field(..., title="Question", description="Question to ask")


yes_no_question_parser = PydanticOutputParser(pydantic_object=YesNoQuestion)

generate_yes_no_question_prompt = """
You'll be given a question. 
1. Rephrase the question to be a yes/no question.
2. Extract a unique word from the question.
3. Create an a yes/no question about the word.

---
Examples:
1.
Main Question: Does god exist?
Word: god
Question: Does it exist?
2.
Main Question: Is abortion ethical?
Word: abortion
Question: Is it ethical?
3. 
Main Question: Is the death penalty ethical?
Word: death penalty
Question: Is it ethical?
4. 
Main Question: Is reality a simulation?
Word: reality
Question: Is it a simulation?
5. 
Main Question: Is there a meaning to life?
Word: life
Question: Is there a meaning to it?
---
{format_instructions}

Here the question
{question}
"""
generate_yes_no_question_prompt = PromptTemplate(
    template=generate_yes_no_question_prompt,
    input_variables=["question"],
    partial_variables={
        "format_instructions": yes_no_question_parser.get_format_instructions(),
    },
)

yes_no_questions = [
    (generate_yes_no_question_prompt | llm | yes_no_question_parser).invoke(
        {"question": question.question}
    )
    for question in all_raw_questions
]

In [11]:
yes_no_questions

[YesNoQuestion(word='reality', question='Is it objective?'),
 YesNoQuestion(word='humans', question='Do they have free will?'),
 YesNoQuestion(word='afterlife', question='Is there one?'),
 YesNoQuestion(word='universal moral code', question='Is there a universal moral code?'),
 YesNoQuestion(word='beauty', question='Is it subjective?'),
 YesNoQuestion(word='time travel', question='Is it possible?'),
 YesNoQuestion(word='universe', question='Is there a purpose to it?'),
 YesNoQuestion(word='consciousness', question='Is it purely physical?'),
 YesNoQuestion(word='fate', question='Is there such a thing as it?'),
 YesNoQuestion(word='ultimate truth', question='Is there an ultimate truth?'),
 YesNoQuestion(word='abortion', question='Is it morally wrong?'),
 YesNoQuestion(word='death penalty', question='Should it be abolished?'),
 YesNoQuestion(word='capitalism', question='Is it the best economic system?'),
 YesNoQuestion(word='government', question='Should it provide universal healthcare?')

In [14]:
answer_yes_no_question_prompt = """
I will ask you a question about a word and you will answer ONLY with a yes or no. Do not mention anything else but a single word answer.
The word is: {word}
My question is: {question}
Your answer:
"""
answer_yes_no_question_prompt = PromptTemplate.from_template(
    template=answer_yes_no_question_prompt,
)

answers = [
    (answer_yes_no_question_prompt | llm | StrOutputParser()).invoke(
        {"word": question.word, "question": question.question}
    )
    for question in yes_no_questions
]

In [15]:
[
    {"word": question.word, "question": question.question, "answer": answer}
    for question, answer in zip(yes_no_questions, answers)
]

[{'word': 'reality', 'question': 'Is it objective?', 'answer': 'No.'},
 {'word': 'humans', 'question': 'Do they have free will?', 'answer': 'Yes.'},
 {'word': 'afterlife', 'question': 'Is there one?', 'answer': 'Yes.'},
 {'word': 'universal moral code',
  'question': 'Is there a universal moral code?',
  'answer': 'No.'},
 {'word': 'beauty', 'question': 'Is it subjective?', 'answer': 'Yes.'},
 {'word': 'time travel', 'question': 'Is it possible?', 'answer': 'No.'},
 {'word': 'universe',
  'question': 'Is there a purpose to it?',
  'answer': 'No.'},
 {'word': 'consciousness',
  'question': 'Is it purely physical?',
  'answer': 'No.'},
 {'word': 'fate',
  'question': 'Is there such a thing as it?',
  'answer': 'Yes.'},
 {'word': 'ultimate truth',
  'question': 'Is there an ultimate truth?',
  'answer': 'Yes.'},
 {'word': 'abortion', 'question': 'Is it morally wrong?', 'answer': 'No.'},
 {'word': 'death penalty',
  'question': 'Should it be abolished?',
  'answer': 'No.'},
 {'word': 'capi

In [16]:
def parse_answer(answer: str) -> bool:
    answer = answer.lower()
    if "yes" in answer:
        return True
    elif "no" in answer:
        return False
    else:
        raise ValueError(f"Invalid answer: {answer}")

parsed_answers = [parse_answer(answer) for answer in answers]

In [31]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

def answer_to_emoji(answer: bool) -> str:
    if answer:
        return "✅"
    else:
        return "❌"
    
rows = [
    (question.word, question.question, answer_to_emoji(answer))
    for question, answer in zip(yes_no_questions, parsed_answers)
]
df = pd.DataFrame(rows, columns=["Word", "Question", "Answer"])
df.set_index("Word", inplace=True, append=False)
df

,Question,Answer
Word,,
reality,Is it objective?,❌
humans,Do they have free will?,✅
afterlife,Is there one?,✅
universal moral code,Is there a universal moral code?,❌
beauty,Is it subjective?,✅
time travel,Is it possible?,❌
universe,Is there a purpose to it?,❌
consciousness,Is it purely physical?,❌
fate,Is there such a thing as it?,✅
